# EPFL BIO 322 Project  
### Gene Prediction based on Mouse brain single cell gene expression profiles

### Authors:

- Simon Lee (simon.lee@epfl.ch) 
- Léa Goffinet (lea.goffinet@epfl.ch)

## Project Description

In an experiment on epigenetics and memory, Giulia Santoni (from the lab of Johannes Gräff at
EPFL) measured the gene expression levels in multiple cells of a mouse brain under three different
conditions that we call KAT5, CBP and eGFP. In this challenge, the goal is to predict – as accurately
as possible – for each cell the experimental condition (KAT5, CBP or eGFP) under which it was
measured, given only the gene expression levels.

In [ ]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell

from tqdm import tqdm
import requests
import os
import sys
from pathlib import Path
import multiprocessing as mp
from multiprocessing import Process
import concurrent
from multiprocessing import Pool
import xgboost as xgb
import glob

InteractiveShell.ast_node_interactivity = "all"

### Data Preprocessing and visualisation

This section covers the loading, filtering and visualisation of the data.

In [ ]:
# read raw data
df = pd.read_csv('../data/train.csv.gz', compression='gzip')
df

As expected single cell data is extremely sparse. Lets count to see how many non zero entries we actually have per column to get a glimpse of what might be important genes and what might not be important.

In [ ]:
gene_column_headers = df.columns.values.tolist()

run this cell only if you want to do filter and write out the counts of how many non zero columns are in each column to a txt file

In [ ]:
# # this gets the counts of each column and drops the column accordingly
# f = open('../data/counts.txt', 'w')
# for gene in gene_column_headers:
#     count = (df[gene] != 0).sum()
    
#     f.write('Counts of gene in cells {} : {} \n'.format(gene, count))  # Uncomment if you want to generate text file with counts

#     # new data generator: Takes 3 hrs to run!!
#     # if count < 500:
#     #     df = df.drop(columns=[gene])

# # df.to_csv('../data/filtered_train.csv.gz, compression='gzip')

# f.close()

In [ ]:
df_filtered = pd.read_csv('../data/filtered_train.csv.gz')

We can see that the filtering process removed over 22,000 genes that were seen across less than 10% of the cells. Though the threshold is a hyperparameter we believe choosing a smaller hyperparameter will be a safer bet to not throw away useful information

In [ ]:
df_filtered

In [ ]:
df_filtered = df_filtered.drop(columns='Unnamed: 0')

In [ ]:
#check for any null values incase we need to perform imputation
check_nan = df.isnull().values.any()
print(check_nan)

In [ ]:
gene_column_headers = df_filtered.columns.values.tolist()

We're performing PCA, once with the whole data and once with the filtered data, to see if there is different clusters but also to make sure that the filtered data is corresponding to the raw data.

In [ ]:
from sklearn.preprocessing import StandardScaler

x = df.iloc[:, :-1].values
y = df.loc[:,['labels']].values

x = StandardScaler().fit_transform(x)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
df_pca = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

In [ ]:
pca.explained_variance_ratio_

In [ ]:
df_pca_final = pd.concat([df_pca, df[['labels']]], axis = 1)

In [ ]:
plt.figure()
plt.figure(figsize=(10,10))

labels = ['CBP', 'KAT5', 'eGFP']
colors = ['r', 'g', 'b']
for label, color in zip(labels,colors):
    indicesToKeep = df_pca_final['labels'] == label
    plt.scatter(df_pca_final.loc[indicesToKeep, 'principal component 1'], df_pca_final.loc[indicesToKeep, 'principal component 2'], c = color, s = 25)
plt.legend(labels)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA : 2 components')
plt.show()

In [ ]:
pca.explained_variance_ratio_
#very low explained variance with 2 components...

In [ ]:
#same with df_filtered

In [ ]:
x = df_filtered.iloc[:, :-1].values
y = df_filtered.loc[:,['labels']].values

x = StandardScaler().fit_transform(x)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
df_pca = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

df_pca_final = pd.concat([df_pca, df[['labels']]], axis = 1)

plt.figure()
plt.figure(figsize=(10,10))

labels = ['CBP', 'KAT5', 'eGFP']
colors = ['r', 'g', 'b']
for label, color in zip(labels,colors):
    indicesToKeep = df_pca_final['labels'] == label
    plt.scatter(df_pca_final.loc[indicesToKeep, 'principal component 1'], df_pca_final.loc[indicesToKeep, 'principal component 2'], c = color, s = 25)
plt.legend(labels)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA : 2 components')
plt.show()

### Method 1 : Multinomial Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [ ]:
X = df_filtered.iloc[:,:-1].values
y = df_filtered['labels'].values

X = StandardScaler().fit_transform(x)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
X_train.shape, y_test.shape

In [ ]:

classifier = LogisticRegression(multi_class='multinomial',solver ='saga')
classifier.fit(X_train, y_train) 

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)


In [ ]:
def plot_confusion_matrix(cm, classes, normalized=True, cmap='bone'):
    norm_cm = cm
    if normalized:
        norm_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(norm_cm, annot=cm, fmt='g', xticklabels=classes, yticklabels=classes, cmap=cmap)

plot_confusion_matrix(cm, ['KAT5', 'CBP', 'eGFP'])

In [ ]:
classifier.score(X_test, y_test)

### Method 2 : Multiple linear regression with L1 regularization

In [ ]:
#mmmmm Logistic is so good, I'm not sure it's worth it to let multiple linear regression, ?

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso

In [ ]:
X = df_filtered.iloc[:,:-1]
y = df_filtered['labels']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)
  
# Splitting the data into training and testing data
model = LinearRegression()
  
model.fit(X_train,y_train)
print(model.score(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
#y_test

In [ ]:
#cm = confusion_matrix(y_test, y_pred)
#plot_confusion_matrix(cm, ['KAT5', 'CBP', 'eGFP'])

In [ ]:
X_train

In [ ]:
coeffs = pd.DataFrame()
coeffs['gene'] = X_train.columns
coeffs['coeffcient'] = pd.Series(model.coef_)
coeffs.tail()

In [ ]:
mean_squared_error_ridge = np.mean((y_pred - y_test)**2)
print(mean_squared_error_ridge)

In [ ]:
lasso = Lasso(alpha = 1)
lasso.fit(X_train, y_train)
y_pred1 = lasso.predict(X_test)

In [ ]:
mean_squared_error = np.mean((y_pred1 - y_test)**2)
print("Mean squared error on test set", mean_squared_error)

In [ ]:
lasso_coeff = pd.DataFrame()
lasso_coeff['gene_column_headers'] = X_train.columns
lasso_coeff['coefficient'] = pd.Series(lasso.coef_)
 
lasso_coeff.head()

## Method 3 XGBoost

- Once with our filtered data
- Also run with raw data to see if filtering has an effect

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
# split into train and validation set using sklearn
gene_column_headers_filtered = gene_column_headers[:-1]
y = df_filtered['labels']
X = df_filtered.iloc[:,:-1]

# need to transform our labels from [KAT5, CBP, eGFP] -> [0,1,2] 
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# perform a typical split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify = y)

training_data = {'X_train':X_train,'y_train':y_train,
                'X_test': X_val,'y_test':y_val}

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# fit function where it takes the sklearn and xgboost models and performs the boosted trees.
# plots performance and accuracy as well

def fit(model, training_data=training_data, epochs=300, label_gene = gene_column_headers_filtered):
    # fitting to the sklearn model
    print('Fitting model...')
    model.fit(training_data['X_train'], training_data['y_train'])
    print('Fitting done!')

    # fitting the xboost library model
    train = xgb.DMatrix(training_data['X_train'], label=training_data['y_train'])
    test = xgb.DMatrix(training_data['X_test'], label=training_data['y_test'])
    params = model.get_xgb_params()
    metrics = ['mlogloss','merror']
    params['eval_metric'] = metrics
    evaluation = {}
    evallist = [(test, 'test'),(train,'train')]
    xgb_model = xgb.train(params, train, epochs, evallist, evals_result=evaluation,verbose_eval=100)

    # Model reports
    print('-- Model Report --')
    print('XGBoost Accuracy: '+str(accuracy_score(model.predict(training_data['X_test']), training_data['y_test'])))
    print('XGBoost F1-Score: '+str(f1_score(model.predict(training_data['X_test']),training_data['y_test'], average='micro')))
    
    # plotting the error curves for our loss functions
    for m in metrics:
        test_score = evaluation['test'][m]
        train_score = evaluation['train'][m]
        x = range(0, epochs)
        plt.rcParams["figure.figsize"] = [6,6]
        plt.plot(x, test_score, label="Test")
        plt.plot(x, train_score, label="Train")
        
        title_name = m + " plot"
        plt.title(title_name)
        plt.xlabel('Epoch')
        plt.ylabel(m)
        lgd = plt.legend()
        plt.show()
    
    # makes sure that the two array match so we can plot the feature importance
    print("length of features list: {}".format(len(gene_column_headers_filtered)))
    print("length of feature importance vector {}".format(len(model.feature_importances_)))

    return xgb_model

In [ ]:
training_data['X_test']

In [ ]:
# hyperparameters that can be adjusted in the final model
xgb_model = XGBClassifier(learning_rate=0.1, # play around with learning rate
                    n_estimators=300, # play around with number of boosted trees built
                    max_depth=9, # play around with tree depth
                    objective='multi:softmax',  # I saw that using softmax or softprob is best for multi class classification
                    nthread=4,
                    num_class=3,
                    seed=1 # seed is included for reproducibility
                    )

xgb_trained = fit(xgb_model, training_data)

read in test.csv.gz to assess model and predictions

In [ ]:
test = pd.read_csv('../data/test.csv.gz', compression='gzip', usecols=gene_column_headers_filtered)
test

In [ ]:
# get it into dmatrix again
X = test
testing_data = xgb.DMatrix(data=X)

In [ ]:
test = xgb.DMatrix(training_data['X_test'], label=training_data['y_test'])
pred = xgb_trained.predict(test)

In [ ]:
print(classification_report(training_data['y_test'], pred))

In [ ]:
confusion_matrix = confusion_matrix(training_data['y_test'], pred)

In [ ]:
import graphviz
plt.rcParams["figure.figsize"] = [22,40]
xgb.plot_tree(xgb_trained)

In [ ]:
plt.rcParams["figure.figsize"] = [22,22]
xgb.plot_importance(xgb_trained)

as expected alot of features aren't very important but lets still try running our classifier with our raw data 

Raw data. Assess performance

In [ ]:
# split into train and validation set using sklearn
y = df['labels']
X = df.loc[:, df.columns != 'labels'] 

# need to transform our labels from [KAT5, CBP, eGFP] -> [0,1,2] 
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# perform a typical split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

data = {'X_train':X_train,'y_train':y_train,
                'X_test': X_val,'y_test':y_val}

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# hyperparameters that can be adjusted in the final model
xgb_model = XGBClassifier(learning_rate=0.1, # play around with learning rate
                    n_estimators=300, # play around with number of boosted trees built
                    max_depth=5, # play around with tree depth
                    objective='multi:softmax',  # I saw that using softmax or softprob is best for multi class classification
                    nthread=4,
                    num_class=3,
                    seed=1 # seed is included for reproducibility
                    )

xgb_trained = fit(xgb_model, training_data)

I kinda 'cleaned' the code a bit, but I don't know if you wanted to include multiple models with different hyperparamaters, cause maybe it's enough to mention in the report that we played around with, or do we let them in ?